# How to use this interactive notebook?

#### 1. Fill in your API key below
#### 2. Press the play button above or select _Run All Cells_ from the _Run_ menu in the toolbar.

In [1]:
API_KEY = "your-api-key-here"

# Use case: The Basics

In this starter use case, we will showcase the fundamentals of the Toqua API and introduce some of the terminology used in our documentation. To get the most out of this tutorial, you should have some familiarity with Python.

We will simulate the performance of a ship in a single weather condition, for a single speed.

## Authorize

Let's first make sure we're authorized to use the API by running the following code.

If everything is alright, you should see a list of ships.


In [2]:
import json
import requests

API_URL = "https://api.toqua.ai"

url = "https://api.toqua.ai/ships/"
headers = {"accept": "application/json", "X-API-Key": API_KEY}
response = requests.get(url, headers=headers)

print(json.dumps(response.json(), indent=2))


[
  {
    "name": "Trial Vessel",
    "imo_number": 9999999,
    "type": "Tanker",
    "class": null,
    "country": "SC",
    "build_year": 2015,
    "shipyard": "Toqua Shipyard",
    "dwt": 220000.0,
    "beam": 55.0,
    "loa": 300.0,
    "mcr": null,
    "max_rpm": null,
    "uuid": "eycrYbrzJNsJecGqKraUCn"
  }
]


Fill in the IMO number of the ship you want to analyze.


In [3]:
IMO_NUMBER = 9999999

## Conditioning parameters

The conditioning parameters are the factors that influence the relation between the ship's primary operational parameters. These can be provided at the input and will influence the value of the predicted primary operational parameters. We can model the impact of any of the following conditions:

### Weather conditions
* Wind speed and direction
* Wave height and direction
* Current speed and direction
* Sea surface salinity
* Sea surface temperature


### Operational conditions
* Mean/average draft
* Trim
* Ship heading
* Fuel specific energy  / Lower Calorific Value

Let's define these conditions.

In [4]:
wind_speed = 6          # [m/s]
wind_direction = 180    # [degrees]
wave_height = 2         # [m]
wave_direction = 90     # [degrees]
current_speed = 0.5     # [m/s]
current_direction = 0   # [degrees]
mean_draft = 20         # [m]
trim = -1               # [m]
ship_heading = 0        # [degrees]
fuel_specific_energy  = 41.5 # [MJ/kg]


## The entrypoint

Now that we have our conditioning parameters, let's specify our entrypoint. 

The entrypoint can be any of the below parameters. Given exactly one of these (the entrypoint), the other parameters can be predicted.
- Speed Over Ground (`sog`)
- Speed Through Water (`stw`)
- Main Engine RPM (`me_rpm`)
- Main Engine Shaft Power (`me_power`)
- Main Engine Fuel Consumption (`me_fo_consumption`).


We want to predict the amount of fuel the ship will consume when sailing at a Speed Over Ground of 13 knots in the conditions we described above.

Thus, our entrypoint will be the `sog` parameter.

In [5]:
sog = 13

## Define the API input


The Toqua API expects the model input to look like this:

```
{
     "date": "...",
     "data": {
          "sog": [...],
          "draft_avg": [...],
          "trim": [...],
          "wave_direction": [...],
          "wave_height": [...],
          "wave_period": [...],
          "current_speed": [...],
          "current_direction": [...],
          "wind_direction": [...],
          "wind_speed": [...],
          "ship_heading": [...],
          "fuel_specific_energy": [...]
     }
}
```

For the full specification, consult our [API reference](https://docs.toqua.ai/reference/retrieve_model_prediction_ships__ship_uuid_or_imo__models_latest_predict_post).


Let's create such an input from the condition and entrypoint we specified. For this use case, we will ignore the `date` parameter.

Note that all parameters have to be provided as lists of values.

In [6]:
api_input = {
    "data": {
        "sog": [sog],
        "wave_direction": [wave_direction],
        "wave_height": [wave_height],
        "wind_direction": [wind_direction],
        "wind_speed": [wind_speed],
        "current_direction": [current_direction],
        "current_speed": [current_speed],
        "draft_avg": [mean_draft],
        "trim": [trim],
        "fuel_specific_energy": [fuel_specific_energy]
    }
}

## Query the API


Now that our input data is in the correct format, let's create a simple function to query the model API and call it.


In [7]:
def query_api(imo_number, payload):
    url = f"https://api.toqua.ai/ships/{imo_number}/models/latest/predict"
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "X-API-Key": API_KEY,
    }
    response = requests.post(url, json=payload, headers=headers)
    return response


If everything went fine, the code below should print `Status Code: 200`.


In [8]:
response = query_api(IMO_NUMBER, api_input)
print("Status Code: ", response.status_code)


Status Code:  200


Let's look at the values that were predicted by the model for the conditions we provided.


In [9]:
print(response.json())

{'sog': [13.0], 'stw': [13.97192], 'me_rpm': [57.87255627071883], 'me_power': [15068.031935400446], 'me_fo_consumption': [63.799937707944665], 'me_fo_emission': [202.27770250303854]}


For the conditions we specified the ship will:
* sail at a STW of 13.97 knots
* require a main engine RPM of 58
* require a main engine power output of 15068 kW
* consume fuel at a rate of 63.8 mt/day
* emit CO2 at a rate of 202 mt/day